In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

## Загрузка данных

In [2]:
#%%capture
#!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [3]:
data = pd.read_excel('data/data_ford_price.xlsx') 

## Отбор признаков: мотивация

### Предобработка данных

In [4]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Отбор признаков: классификация методов

###Метод рекурсивного исключения признаков

In [5]:
from sklearn.feature_selection import RFE

In [6]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

Приоритетные признаки 'year', 'cylinders' и 'lat'.

In [7]:
data_RFE = data[['year', 'cylinders', 'lat']].copy()

y = data['price']
x = data_RFE

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Обучение модели на 3 столбцах по методу RFE

In [8]:
model_RFE= LinearRegression()

model_RFE.fit(X_train, y_train)
y_predicted_RFE = model_RFE.predict(X_test)
 
mae_RFE = mean_absolute_error(y_test, y_predicted_RFE)
R2_RFE = r2_score(y_test, y_predicted_RFE)
print('MAE RFE: %.3f' % mae_RFE)
print('R2_RFE: %.3f' % R2_RFE)

MAE RFE: 5280.017
R2_RFE: 0.562


## МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression

In [10]:
y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [12]:
data_pearson = data[['year', 'cylinders', 'odometer']].copy()

y = data['price']
x = data_pearson

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Обучение модели на 3 столбцах по методу Pearson`s

In [13]:
model_pearson= LinearRegression()

model_pearson.fit(X_train, y_train)
y_predicted_pearson = model_pearson.predict(X_test)
 
mae_pearson = mean_absolute_error(y_test, y_predicted_pearson)
R2_pearson = r2_score(y_test, y_predicted_pearson)
print('MAE Pearson: %.3f' % mae_pearson)
print('R2_Pearson: %.3f' % R2_pearson)

MAE Pearson: 4925.458
R2_Pearson: 0.598


# **ВЫВОД**

Метрики линейной регрессии на признаках, отобранных по методу рекурсивного исключения (RFE):
+ MAE RFE: 5280.017
+ R2_RFE: 0.562

Метрики линейной регресии на признаках отобранных по коэффициенту корреляции Пирсона:
+ MAE Pearson: 4925.458
+ R2_Pearson: 0.598

## По результатам метрик видим, что значимость признаков по коэффициенту корреляци Пирсона лучше влияют на модель по абсолютной ошибке MAE и коэффициенту детерминаци.